# Homework and bake-off: word-level entailment with neural networks

In [15]:
__author__ = "Christopher Potts"
__version__ = "CS224u, Stanford, Fall 2020"

## Contents

1. [Overview](#Overview)
1. [Set-up](#Set-up)
1. [Data](#Data)
1. [Baseline](#Baseline)
  1. [Representing words: vector_func](#Representing-words:-vector_func)
  1. [Combining words into inputs: vector_combo_func](#Combining-words-into-inputs:-vector_combo_func)
  1. [Classifier model](#Classifier-model)
  1. [Baseline results](#Baseline-results)
1. [Homework questions](#Homework-questions)
  1. [Hypothesis-only baseline [2 points]](#Hypothesis-only-baseline-[2-points])
  1. [Alternatives to concatenation [2 points]](#Alternatives-to-concatenation-[2-points])
  1. [A deeper network [2 points]](#A-deeper-network-[2-points])
  1. [Your original system [3 points]](#Your-original-system-[3-points])
1. [Bake-off [1 point]](#Bake-off-[1-point])

## Overview

The general problem is word-level natural language inference. Training examples are pairs of words $(w_{L}, w_{R}), y$ with $y = 1$ if $w_{L}$ entails $w_{R}$, otherwise $0$.

The homework questions below ask you to define baseline models for this and develop your own system for entry in the bake-off, which will take place on a held-out test-set distributed at the start of the bake-off. (Thus, all the data you have available for development is available for training your final system before the bake-off begins.)

## Set-up

See [the first notebook in this unit](nli_01_task_and_data.ipynb) for set-up instructions.

In [16]:
from collections import defaultdict
import json
import numpy as np
import os
import pandas as pd
from torch_shallow_neural_classifier import TorchShallowNeuralClassifier
import nli
import utils

In [17]:
DATA_HOME = 'data'

NLIDATA_HOME = os.path.join(DATA_HOME, 'nlidata')

wordentail_filename = os.path.join(
    NLIDATA_HOME, 'nli_wordentail_bakeoff_data.json')

GLOVE_HOME = os.path.join(DATA_HOME, 'glove.6B')

## Data

I've processed the data into a train/dev split that is designed to put some pressure on our models to actually learn these semantic relations, as opposed to exploiting regularities in the sample. 

The defining feature of the dataset is that the `train` and `dev` __vocabularies__ are disjoint. That is, if a word `w` appears in a training pair, it does not occur in any text pair. It follows from this that there are also no word-pairs shared between train and dev, as you would expect. This should require your models to learn abstract relationships, as opposed to memorizing incidental properties of individual words in the dataset.

In [18]:
with open(wordentail_filename) as f:
    wordentail_data = json.load(f)

The keys are the splits plus a list giving the vocabulary for the entire dataset:

In [19]:
wordentail_data.keys()

dict_keys(['dev', 'train', 'vocab'])

In [20]:
wordentail_data['train'][: 5]

[[['abode', 'house'], 1],
 [['abortion', 'anaemia'], 0],
 [['abortion', 'aneurysm'], 0],
 [['abortion', 'blindness'], 0],
 [['abortion', 'deafness'], 0]]

In [21]:
nli.get_vocab_overlap_size(wordentail_data)

0

Because no words are shared between `train` and `dev`, no pairs are either:

In [22]:
nli.get_pair_overlap_size(wordentail_data)

0

Here is the label distribution:

In [23]:
pd.DataFrame(wordentail_data['train'])[1].value_counts()

0    7000
1    1283
Name: 1, dtype: int64

This is a challenging label distribution – there are more than 5 times as more non-entailment cases as entailment cases.

## Baseline

Even in deep learning, __feature representation is vital and requires care!__ For our task, feature representation has two parts: representing the individual words and combining those representations into a single network input.

### Representing words: vector_func

Let's consider two baseline word representations methods:

1. Random vectors (as returned by `utils.randvec`).
1. 50-dimensional GloVe representations.

In [24]:
def randvec(w, n=50, lower=-1.0, upper=1.0):
    """Returns a random vector of length `n`. `w` is ignored."""
    return utils.randvec(n=n, lower=lower, upper=upper)

In [25]:
def load_glove50():
    glove_src = os.path.join(GLOVE_HOME, 'glove.6B.50d.txt')
    # Creates a dict mapping strings (words) to GloVe vectors:
    GLOVE = utils.glove2dict(glove_src)
    return GLOVE

GLOVE = load_glove50()

def glove_vec(w):
    """Return `w`'s GloVe representation if available, else return
    a random vector."""
    return GLOVE.get(w, randvec(w, n=50))

### Combining words into inputs: vector_combo_func

Here we decide how to combine the two word vectors into a single representation. In more detail, where `u` is a vector representation of the left word and `v` is a vector representation of the right word, we need a function `vector_combo_func` such that `vector_combo_func(u, v)` returns a new input vector `z` of dimension `m`. A simple example is concatenation:

In [26]:
def vec_concatenate(u, v):
    """Concatenate np.array instances `u` and `v` into a new np.array"""
    return np.concatenate((u, v))

`vector_combo_func` could instead be vector average, vector difference, etc. (even combinations of those) – there's lots of space for experimentation here; [homework question 2](#Alternatives-to-concatenation-[2-points]) below pushes you to do some exploration.

### Classifier model

For a baseline model, I chose `TorchShallowNeuralClassifier`:

In [27]:
net = TorchShallowNeuralClassifier(early_stopping=True)

### Baseline results

The following puts the above pieces together, using `vector_func=glove_vec`, since `vector_func=randvec` seems so hopelessly misguided for our problem!

In [28]:
baseline_experiment = nli.wordentail_experiment(
    train_data=wordentail_data['train'],
    assess_data=wordentail_data['dev'],
    model=net,
    vector_func=glove_vec,
    vector_combo_func=vec_concatenate)

Stopping after epoch 140. Validation score did not improve by tol=1e-05 for more than 10 epochs. Final error is 1.023185983300209

              precision    recall  f1-score   support

           0      0.870     0.950     0.908      1732
           1      0.506     0.263     0.346       334

    accuracy                          0.839      2066
   macro avg      0.688     0.607     0.627      2066
weighted avg      0.811     0.839     0.818      2066



## Homework questions

Please embed your homework responses in this notebook, and do not delete any cells from the notebook. (You are free to add as many cells as you like as part of your responses.)

### Hypothesis-only baseline [2 points]

During our discussion of SNLI and MultiNLI, we noted that a number of research teams have shown that hypothesis-only baselines for NLI tasks can be remarkably robust. This question asks you to explore briefly how this baseline affects our task.

For this problem, submit two functions:

1. A `vector_combo_func` function called `hypothesis_only` that simply throws away the premise, using the unmodified hypothesis (second) vector as its representation of the example.

1. A function called `run_hypothesis_only_evaluation` that does the following:
    1. Loops over the two `vector_combo_func` values `vec_concatenate` and `hypothesis_only`, calling `nli.wordentail_experiment` to train on the 'train' portion and assess on the 'dev' portion, with `glove_vec` as the `vector_func`. So that the results are consistent, use an `sklearn.linear_model.LogisticRegression` with default parameters as the model.
    1. Returns a `dict` mapping `function_name` strings to the 'macro-F1' score for that pair, as returned by the call to `nli.wordentail_experiment`. (Tip: you can get the `str` name of, e.g., `hypothesis_only` with `hypothesis_only.__name__`.)
    
The functions `test_hypothesis_only` and `test_run_hypothesis_only_evaluation` will help ensure that your functions have the desired logic.

In [29]:
def hypothesis_only(u, v):
    ##### YOUR CODE HERE
    return v

def run_hypothesis_only_evaluation():
    ##### YOUR CODE HERE
    from sklearn.linear_model import LogisticRegression
    result = dict()
    lr = LogisticRegression()
    for combo in [hypothesis_only, vec_concatenate]:
        experiment = nli.wordentail_experiment(
            train_data=wordentail_data['train'],
            assess_data=wordentail_data['dev'],
            model=lr,
            vector_func=glove_vec,
            vector_combo_func=combo)
        result[combo.__name__] = experiment['macro-F1']
    return result


In [30]:
def test_hypothesis_only(hypothesis_only):
    v = hypothesis_only(1, 2)
    assert v == 2

In [31]:
test_hypothesis_only(hypothesis_only)

In [32]:
def test_run_hypothesis_only_evaluation(run_hypothesis_only_evaluation):
    results = run_hypothesis_only_evaluation()
    assert all(x in results for x in ('hypothesis_only', 'vec_concatenate')), \
        ("The return value of `run_hypothesis_only_evaluation` does not "
         "have the intended kind of keys.")
    assert isinstance(results['vec_concatenate'], float), \
        ("The values of the `run_hypothesis_only_evaluation` result "
         "should be floats.")

In [33]:
test_run_hypothesis_only_evaluation(run_hypothesis_only_evaluation)

              precision    recall  f1-score   support

           0      0.853     0.971     0.908      1732
           1      0.462     0.129     0.201       334

    accuracy                          0.835      2066
   macro avg      0.657     0.550     0.555      2066
weighted avg      0.789     0.835     0.794      2066

              precision    recall  f1-score   support

           0      0.863     0.954     0.906      1732
           1      0.473     0.213     0.293       334

    accuracy                          0.834      2066
   macro avg      0.668     0.583     0.600      2066
weighted avg      0.800     0.834     0.807      2066



### Alternatives to concatenation [2 points]

We've so far just used vector concatenation to represent the premise and hypothesis words. This question asks you to explore two simple alternative:

1. Write a function `vec_diff` that, for a given pair of vector inputs `u` and `v`, returns the element-wise difference between `u` and `v`.

1. Write a function `vec_max` that, for a given pair of vector inputs `u` and `v`, returns the element-wise max values between `u` and `v`.

You needn't include your uses of `nli.wordentail_experiment` with these functions, but we assume you'll be curious to see how they do!

In [34]:
def vec_diff(u, v):
    ##### YOUR CODE HERE
    return u - v


def vec_max(u, v):
    ##### YOUR CODE HERE
    return np.maximum(u, v)



In [35]:
def test_vec_diff(vec_diff):
    u = np.array([10.2, 8.1])
    v = np.array([1.2, -7.1])
    result = vec_diff(u, v)
    expected = np.array([9.0, 15.2])
    assert np.array_equal(result, expected), \
        "Expected {}; got {}".format(expected, result)

In [36]:
test_vec_diff(vec_diff)

In [37]:
def test_vec_max(vec_max):
    u = np.array([1.2,  8.1])
    v = np.array([10.2, -7.1])
    result = vec_max(u, v)
    expected = np.array([10.2, 8.1])
    assert np.array_equal(result, expected), \
        "Expected {}; got {}".format(expected, result)

In [38]:
test_vec_max(vec_max)

### A deeper network [2 points]

It is very easy to subclass `TorchShallowNeuralClassifier` if all you want to do is change the network graph: all you have to do is write a new `build_graph`. If your graph has new arguments that the user might want to set, then you should also redefine `__init__` so that these values are accepted and set as attributes.

For this question, please subclass `TorchShallowNeuralClassifier` so that it defines the following graph:

$$\begin{align}
h_{1} &= xW_{1} + b_{1} \\
r_{1} &= \textbf{Bernoulli}(1 - \textbf{dropout_prob}, n) \\
d_{1} &= r_1 * h_{1} \\
h_{2} &= f(d_{1}) \\
h_{3} &= h_{2}W_{2} + b_{2}
\end{align}$$

Here, $r_{1}$ and $d_{1}$ define a dropout layer: $r_{1}$ is a random binary vector of dimension $n$, where the probability of a value being $1$ is given by $1 - \textbf{dropout_prob}$. $r_{1}$ is multiplied element-wise by our first hidden representation, thereby zeroing out some of the values. The result is fed to the user's activation function $f$, and the result of that is fed through another linear layer to produce $h_{3}$. (Inside `TorchShallowNeuralClassifier`, $h_{3}$ is the basis for a softmax classifier; no activation function is applied to it because the softmax scaling is handled internally by the loss function.)

For your implementation, please use `nn.Sequential`, `nn.Linear`, and `nn.Dropout` to define the required layers.

For comparison, using this notation, `TorchShallowNeuralClassifier` defines the following graph:

$$\begin{align}
h_{1} &= xW_{1} + b_{1} \\
h_{2} &= f(h_{1}) \\
h_{3} &= h_{2}W_{2} + b_{2}
\end{align}$$

The following code starts this sub-class for you, so that you can concentrate on `build_graph`. Be sure to make use of `self.dropout_prob`.

For this problem, submit just your completed  `TorchDeepNeuralClassifier`. You needn't evaluate it, though we assume you will be keen to do that!

You can use `test_TorchDeepNeuralClassifier` to ensure that your network has the intended structure.

In [39]:
import torch.nn as nn

class TorchDeepNeuralClassifier(TorchShallowNeuralClassifier):
    def __init__(self, dropout_prob=0.7, **kwargs):
        self.dropout_prob = dropout_prob
        super().__init__(**kwargs)

    def build_graph(self):
        """Complete this method!

        Returns
        -------
        an `nn.Module` instance, which can be a free-standing class you
        write yourself, as in `torch_rnn_classifier`, or the outpiut of
        `nn.Sequential`, as in `torch_shallow_neural_classifier`.

        """
        ##### YOUR CODE HERE
        return nn.Sequential(
            nn.Linear(self.input_dim, self.hidden_dim),
            nn.Dropout(p=self.dropout_prob),
            self.hidden_activation,
            nn.Linear(self.hidden_dim, self.n_classes_))



In [40]:
def test_TorchDeepNeuralClassifier(TorchDeepNeuralClassifier):
    dropout_prob = 0.55
    assert hasattr(TorchDeepNeuralClassifier(), "dropout_prob"), \
        "TorchDeepNeuralClassifier must have an attribute `dropout_prob`."
    try:
        inst = TorchDeepNeuralClassifier(dropout_prob=dropout_prob)
    except TypeError:
        raise TypeError("TorchDeepNeuralClassifier must allow the user "
                        "to set `dropout_prob` on initialization")
    inst.input_dim = 10
    inst.n_classes_ = 5
    graph = inst.build_graph()
    assert len(graph) == 4, \
        "The graph should have 4 layers; yours has {}".format(len(graph))
    expected = {
        0: 'Linear',
        1: 'Dropout',
        2: 'Tanh',
        3: 'Linear'}
    for i, label in expected.items():
        name = graph[i].__class__.__name__
        assert label in name, \
            ("The {} layer of the graph should be a {} layer; "
            "yours is {}".format(i, label, name))
    assert graph[1].p == dropout_prob, \
        ("The user's value for `dropout_prob` should be the value of "
         "`p` for the Dropout layer.")

In [41]:
test_TorchDeepNeuralClassifier(TorchDeepNeuralClassifier)

### Your original system [3 points]

This is a simple dataset, but its "word-disjoint" nature ensures that it's a challenging one, and there are lots of modeling strategies one might adopt. 

You are free to do whatever you like. We require only that your system differ in some way from those defined in the preceding questions. They don't have to be completely different, though. For example, you might want to stick with the model but represent examples differently, or the reverse.

You are free to use different pretrained word vectors and the like.

Please embed your code in this notebook so that we can rerun it.

In the cell below, please provide a brief technical description of your original system, so that the teaching team can gain an understanding of what it does. This will help us to understand your code and analyze all the submissions to identify patterns and strategies.  We also ask that you report the best score your system got during development, just to help us understand how systems performed overall.

In [69]:
# PLEASE MAKE SURE TO INCLUDE THE FOLLOWING BETWEEN THE START AND STOP COMMENTS:
#   1) Textual description of your system.
#   2) The code for your original system.
#   3) The score achieved by your system in place of MY_NUMBER.
#        With no other changes to that line.
#        You should report your score as a decimal value <=1.0
# PLEASE MAKE SURE NOT TO DELETE OR EDIT THE START AND STOP COMMENTS

# NOTE: MODULES, CODE AND DATASETS REQUIRED FOR YOUR ORIGINAL SYSTEM 
# SHOULD BE ADDED BELOW THE 'IS_GRADESCOPE_ENV' CHECK CONDITION. DOING
# SO ABOVE THE CHECK MAY CAUSE THE AUTOGRADER TO FAIL.

# START COMMENT: Enter your system description in this cell.
# The system uses:
#    The Glove 50 embedded space used previously as we didn't find any significant improvement in using other 
# embedded spaces. 
#
#    vec_concatenate to combine the 2 vector words in a single representation. If the hypothesis belongs to the 
# premise hypernyms, hyponym, member_holonyms, antonyms, derivationally_related_forms, pertainyms as defined 
# by Wordnet, we translate the relevant representation by a value which is halfway between the 
# mean of the sums of the rows of the Glove embedded space and its max. For that purpose we split evenly a 50 array 
# between these features. 
# 
# We have then a list of 3 arrays of shape (50,), one for the premise, one for the 
# hypothesis and one for the different wordnet hierarchies considered. 
#
#  Finally, we use a LSTM RNN neural network classifier defined in the class TorchRNNClassifier. 
#
# My peak score was: 0.708

if 'IS_GRADESCOPE_ENV' not in os.environ:
    from nltk.corpus import wordnet as wn
    import torch.nn as nn
    from sklearn.metrics import classification_report
    from torch_rnn_classifier import TorchRNNClassifier
    
    utils.fix_random_seeds()
    
    
    def wordnet_features(word1, word2, methodname):
        """
        Returns 1 if the synsets extracted from the hierarchy methodname of the premise intersect the synsets of the 
        hypothesis. Otherwise returns -1
        :param word1: word in premise
        :param word2: word in hypothesis
        :param methodname: wordnet hierarchy considered (hypernym, hyponym, member_holonyms, antonyms,
                                                        derivationally_related_forms, pertainyms)
        :return:
        """
        try:
            hyps = [h for ss in wn.synsets(word1) for h in getattr(ss, methodname)()]
        except AttributeError:
            hyps = list()
            for ss in wn.synsets(word1):
                try:
                    for h in getattr(ss.lemmas()[0], methodname)():
                        hyps.append(h)
                except TypeError:
                    pass
        syns = wn.synsets(word2)

        output = 1 if set(hyps) & set(syns) else -1
        return output
    
    def word_entail_featurize_wordnet_1(data, vector_func, vector_combo_func, hierarchy, adjustment_param, rnn):
        """
        Modified version of the featurize function used in nli.wordentail_experiment. Uses WordNet hierarchy.
        :param data:
        :param vector_func:
        :param vector_combo_func:
        :param hierarchy: hierarchy in wordnet considered, 'hypernyms', 'hyponyms', 'member_holonyms', 'antonyms',
                         'derivationally_related_forms', 'pertainyms'
        :param adjustment_param: Parameters used to calculate the featurized vector used to represent if
        the hypothesis is included in the wordnet hierarchy of the premise.
        :return:
        """
        X = []
        y = []
        for (w1, w2), label in data:
            is_related = list()
            for h in hierarchy:
                is_related.append(wordnet_features(w1, w2, h))
            rep = vector_combo_func(vector_func(w1), vector_func(w2))

            # build a vector of length vector_func(w1) with info from wordnet
            vec_size = vector_func(w1).shape[0]
            hierarchy_size = vec_size // len(hierarchy)
            coeff_vec = np.array([x for item in is_related for x in [item] * hierarchy_size])
            remaining_items = vec_size % len(hierarchy)
            if remaining_items > 0:
                # makes sure that coeff_vec has the right size
                coeff_vec = np.concatenate((coeff_vec, np.zeros(remaining_items)))
            adj_vec = coeff_vec * adjustment_param

            rep = np.concatenate((rep, adj_vec))
            if rnn:
                rep = [rep[0:50], rep[50:100], rep[100:]]
            X.append(rep)
            y.append(label)
        return X, y

    def word_entail_featurize_wordnet(hierarchy, param, rnn):
        """
        Wrapper of word_entail_featurize_wordnet_1, allowing to define hierarchy, adjustment_param and rnn.
        in the feature function.
        """
        return lambda *args, **kwargs: word_entail_featurize_wordnet_1(*args, hierarchy=hierarchy,
                                                                   adjustment_param=param, rnn=rnn, **kwargs)
    
    def nli_model():
        """
        Original system
        """
        glove_dic = GLOVE
        df = pd.DataFrame.from_dict(glove_dic, orient='index')
        sum_rows = df.sum(axis=1)

        # calculate param, average distance between the mean and the max/min sum of row values of the Glove embedded space.
        # will be used in word_entail_featurize_wordnet to adjust vectors when they share an hypernym.
        n = df.shape[1]
        mean, max1, min1 = sum_rows.mean() / n, sum_rows.max() / n, sum_rows.min() / n
        param = 0.25 * (abs(min1) + abs(max1)) + mean

        featurize_func = word_entail_featurize_wordnet(['hypernyms', 'hyponyms', 'member_holonyms', 'antonyms', 
                                                        'derivationally_related_forms', 'pertainyms'], 
                                                        param, rnn=True)

        model = TorchRNNClassifier(vocab=[], use_embedding=False, early_stopping=True, rnn_cell_class=nn.LSTM,
                                   hidden_dim=50, batch_size=30, eta=0.05, bidirectional=True, l2_strength=0.001)

        X_train, y_train = featurize_func(wordentail_data['train'], glove_vec, vec_concatenate)
        X_dev, y_dev = featurize_func(wordentail_data['dev'], glove_vec, vec_concatenate)
        model.fit(X_train, y_train)
        predictions = model.predict(X_dev)
        # Report:
        print(classification_report(y_dev, predictions, digits=3))
        macrof1 = utils.safe_macro_f1(y_dev, predictions)
        return {
            'model': model,
            'train_data': wordentail_data['train'],
            'assess_data': wordentail_data['dev'],
            'macro-F1': macrof1,
            'vector_func': glove_vec,
            'vector_combo_func': vec_concatenate}
    
    model_result = nli_model()

# STOP COMMENT: Please do not remove this comment.

Stopping after epoch 17. Validation score did not improve by tol=1e-05 for more than 10 epochs. Final error is 78.43059753254056

              precision    recall  f1-score   support

           0      0.898     0.934     0.915      1732
           1      0.566     0.449     0.501       334

    accuracy                          0.855      2066
   macro avg      0.732     0.691     0.708      2066
weighted avg      0.844     0.855     0.848      2066



## Bake-off [1 point]

The goal of the bake-off is to achieve the highest __macro-average F1__ score on a test set that we will make available at the start of the bake-off. The announcement will go out on the discussion forum. To enter, you'll be asked to run `nli.bake_off_evaluation` on the output of your chosen `nli.wordentail_experiment` run. 

The cells below this one constitute your bake-off entry.

The rules described in the [Your original system](#Your-original-system-[3-points]) homework question are also in effect for the bake-off.

Systems that enter will receive the additional homework point, and systems that achieve the top score will receive an additional 0.5 points. We will test the top-performing systems ourselves, and only systems for which we can reproduce the reported results will win the extra 0.5 points.

Late entries will be accepted, but they cannot earn the extra 0.5 points. Similarly, you cannot win the bake-off unless your homework is submitted on time.

The announcement will include the details on where to submit your entry.

In [43]:
# Enter your bake-off assessment code into this cell.
# Please do not remove this comment.
if 'IS_GRADESCOPE_ENV' not in os.environ:
    pass
    # Please enter your code in the scope of the above conditional.
    ##### YOUR CODE HERE


In [44]:
# On an otherwise blank line in this cell, please enter
# your macro-avg f1 value as reported by the code above.
# Please enter only a number between 0 and 1 inclusive.
# Please do not remove this comment.
if 'IS_GRADESCOPE_ENV' not in os.environ:
    pass
    # Please enter your score in the scope of the above conditional.
    ##### YOUR CODE HERE
